## 학습목표
- 네이버카페 url 패턴 분석 (사용자에게 입력받아) 하여 기간 설정값을 변경하여 데이터를 수집

- 실습순서
1. 네이버 카페 url 분석하기
2. 검색할 키워드와 기간설정값을 변수로 만든 후 url 적용하기
3. selenium을 활용한 브라우저 띄우기
4. 스크롤 내리기
5. 조회된 카페 게시글 url수집하여 리스트 저장
6. 리스트 내의 게시글 요청하여 데이터 수집
7. 전처리 후 텍스트파일에 저장

In [3]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from kiwipiepy import Kiwi
from wordcloud import WordCloud as wc
from collections import Counter
from tqdm import tqdm
import re

In [4]:
# 문자열 전처리 함수 -> 숫자, 문자, (?,!,) 제외하고 공백으로 대체
def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,]+"," ", w)
    w = w.strip()
    return w

- 글(문자열) 데이터가 url에 들어갔을 때 띄어쓰기가 있으면 url 오류가 뜸
- url에 글이 들어갔을 때 안전하게 사용할 수 있는 형태로 변경해주는 기능 사용


In [5]:
from urllib.parse import quote

In [7]:
keyword = quote('음식물 처리기 사용후기')

In [15]:
# date_option 패턴: 0(전체), 1(1시간),2(1일),3(1주),4(1개월),5(3개월),6(6개월), 7(1년)
date_option = 6
url = f"https://search.naver.com/search.naver?cafe_where=&date_option={date_option}&prdtype=0&query={keyword}&sm=mtb_opt&ssc=tab.cafe.all&st=rel&stnm=rel&opt_tab=0&nso=so%3Ar%2Cp%3A1y%276m"

- 결국 뒷자리의 6m가 6개월을 의미하는 것이 아니라, "date_option="값이 기간을 결정함!

- 키워드와 date_option만 변경하면 원하는 키워드와 원하는 기간을 임의로 설정해서 사용자에게 받아서 크롤링할 수 있음!

In [16]:
url

'https://search.naver.com/search.naver?cafe_where=&date_option=6&prdtype=0&query=%EC%9D%8C%EC%8B%9D%EB%AC%BC%20%EC%B2%98%EB%A6%AC%EA%B8%B0%20%EC%82%AC%EC%9A%A9%ED%9B%84%EA%B8%B0&sm=mtb_opt&ssc=tab.cafe.all&st=rel&stnm=rel&opt_tab=0&nso=so%3Ar%2Cp%3A1y%276m'

In [64]:
driver = wb.Chrome()
driver.get(url)

In [41]:
scroll = driver.find_element(By.TAG_NAME,'body')

In [42]:
try:
    for i in range(10):
        scroll.send_keys(Keys.END)
        time.sleep(2)
except:
    pass

In [43]:
len(driver.find_elements(By.CSS_SELECTOR,'a.title_link'))

330

In [44]:
link = driver.find_elements(By.CSS_SELECTOR,'a.title_link')

In [45]:
href_list = [l.get_attribute('href') for l in link]

In [65]:
len(href_list)

330

In [94]:
driver.get(href_list[1])

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6772988d5
	0x7ff677298930
	0x7ff677071465
	0x7ff6770b9d83
	0x7ff6770f1ee2
	0x7ff6770ec500
	0x7ff6770eb6f0
	0x7ff67703b325
	0x7ff6775b0620
	0x7ff6775aaf60
	0x7ff6775c96c6
	0x7ff6772b5dd4
	0x7ff6772bed7c
	0x7ff677039cb8
	0x7ff67770efe8
	0x7ffbdaa7e8d7
	0x7ffbdba4c53c


- iframe의 개념이 생각나지 않았음!! > user-agent를 가져오는 것은 request 400 즉, 접근제한이 걸렸을때의 문제였음! 헷갈리지 말자!

In [95]:
driver = wb.Chrome()
driver.get(src)

In [104]:
src = driver.find_element(By.ID,'cafe_main')

In [105]:
src

<selenium.webdriver.remote.webelement.WebElement (session="912eca12dfda7dee918f40e07e711674", element="f.A179BA92C3B65CBCB90146C6CF7B1EDE.d.ABD1AD2952702816CF4257802867E212.e.139")>

In [106]:
driver.switch_to.frame(src)

In [108]:
driver.find_elements(By.CSS_SELECTOR,'div.se-main-container')[0].text

'다이렉트결혼준비를 소개하면서 다이렉트결혼준비로부터 웨딩소품(부케, 폐백 등) 등을 구입할 수 있는 포인트를 받을 수 있었음.\n안녕하세요, 9월 예신입니다\n우리집 미니 가전인 미닉스 음식물처리기를 소개합니다!\n저는 원래 음식물쓰레기 냉장고를 사용했었는데요,\n신혼집으로 이사오면서 기존 냉장고를 처분하고 그 유명한 미닉스 음식물처리기를 구매했습니다!\n최근 출시된 max는 3L까지 음식물을 저장할 수 있는 큰 용량을 가지고 있어서 좋았어요 ㅎㅎ 둘이서 사용하기 충분합니다\nAuto모드가 있어서 자동으로 음식물을 저장했다가 때가되면 음식물 처리를 해줘요!\n전용 필터가 있어서 음식물 처리가 되기 전이어도 냄새가 나지않고, 신내가 슬슬 나기 시작하면 필터를 갈아주면 됩니다 ㅎㅎ\n뚜껑을 연 모습이에요!\n다 좋은데, 관리하기가 좀 힘들어요 ㅠㅠ\n다 처리된 음식물을 버리는거나, 안에 있는 용기를 빼면 언제 들어갔는지 안쪽에 부스러기가 들어가있을때가 있더라구요 ㅜ 그런거 청소하기 좀 힘들어요 ••\n그리고 수분기 있는거나 매운 음식(?) 같은거 넣으면 좀 꾸덕해지는데 그것도 처리하기가 좀 힘들어서 물에 한번 씻고 넣어주던지 해야하는게 번거로워요 ㅠㅠ\n그래도 쓰레기 냄새 안나고, 음식물쓰레기 버리러가는 빈도가 줄은 것만으로도 넘 만족하며 사용하고 있는 가전입니다 ~!!\n음식물처리기 고민하고계신 분들께 추천드려요 !!'

In [109]:
preprocess_sentence_kr(driver.find_elements(By.CSS_SELECTOR,'div.se-main-container')[0].text)

'다이렉트결혼준비를 소개하면서 다이렉트결혼준비로부터 웨딩소품 부케, 폐백 등 등을 구입할 수 있는 포인트를 받을 수 있었음. 안녕하세요, 9월 예신입니다 우리집 미니 가전인 미닉스 음식물처리기를 소개합니다! 저는 원래 음식물쓰레기 냉장고를 사용했었는데요, 신혼집으로 이사오면서 기존 냉장고를 처분하고 그 유명한 미닉스 음식물처리기를 구매했습니다! 최근 출시된 는 3 까지 음식물을 저장할 수 있는 큰 용량을 가지고 있어서 좋았어요 둘이서 사용하기 충분합니다 모드가 있어서 자동으로 음식물을 저장했다가 때가되면 음식물 처리를 해줘요! 전용 필터가 있어서 음식물 처리가 되기 전이어도 냄새가 나지않고, 신내가 슬슬 나기 시작하면 필터를 갈아주면 됩니다 뚜껑을 연 모습이에요! 다 좋은데, 관리하기가 좀 힘들어요 다 처리된 음식물을 버리는거나, 안에 있는 용기를 빼면 언제 들어갔는지 안쪽에 부스러기가 들어가있을때가 있더라구요 그런거 청소하기 좀 힘들어요 그리고 수분기 있는거나 매운 음식 ? 같은거 넣으면 좀 꾸덕해지는데 그것도 처리하기가 좀 힘들어서 물에 한번 씻고 넣어주던지 해야하는게 번거로워요 그래도 쓰레기 냄새 안나고, 음식물쓰레기 버리러가는 빈도가 줄은 것만으로도 넘 만족하며 사용하고 있는 가전입니다 !! 음식물처리기 고민하고계신 분들께 추천드려요 !!'

In [112]:
driver.get(href_list[0])
src = driver.find_element(By.ID,'cafe_main')
driver.switch_to.frame(src)


In [116]:
preprocess_sentence_kr(driver.find_elements(By.CSS_SELECTOR,'div.se-main-container')[0].text)

'다이렉트결혼준비를 소개하면서 다이렉트결혼준비로부터 웨딩소품 부케, 폐백 등 등을 구입할 수 있는 포인트를 받을 수 있었음 쿠쿠 건조분쇄형 에코웨일 2 음식물처리기 그레이스화이트 201 사용후기입니다. 신혼집에 설치해서 사용 중인 쿠쿠 건조분쇄형 에코웨일 2 음식물처리기입니다. 맞벌이라 음식물 쓰레기 버리는 게 생각보다 번거로웠는데, 이 제품 들이고 나서 정말 삶의 질이 달라졌어요. 음식물 냄새 걱정 없이 바로 처리할 수 있어서 주방이 항상 깔끔하게 유지됩니다. 사이즈도 컴팩트해서 주방 상판 위에 올려두어도 자리 차지를 많이 하지 않고, 그레이스화이트 컬러라 신혼집 인테리어에도 잘 어울려요. 건조 후 분쇄까지 한 번에 되니까 부피가 확 줄어들고, 처리 후 냄새도 거의 없어 만족스럽습니다. 퇴근 후 설거지하면서 음식물 바로 넣고 버튼만 누르면 끝이라 정말 편하고, 작동 시간도 길지 않아서 부담 없어요. 맞벌이 신혼부부나 음식물 쓰레기 처리 스트레스 줄이고 싶은 분들께 꼭 추천하고 싶은 가전입니다. 음식물처리기는 신혼부부 필수제품 인 거 같아용 추천합니다'

In [115]:
driver.find_elements(By.CSS_SELECTOR,'div.se-main-container')[0].text

"'다이렉트결혼준비를 소개하면서 다이렉트결혼준비로부터 웨딩소품(부케, 폐백 등) 등을 구입할 수 있는 포인트를 받을 수 있었음‘\n쿠쿠 건조분쇄형 에코웨일 2L 음식물처리기 그레이스화이트 CFD-ENL201DCGW 사용후기입니다.\n신혼집에 설치해서 사용 중인 쿠쿠 건조분쇄형 에코웨일 2L 음식물처리기입니다. 맞벌이라 음식물 쓰레기 버리는 게 생각보다 번거로웠는데, 이 제품 들이고 나서 정말 삶의 질이 달라졌어요. 음식물 냄새 걱정 없이 바로 처리할 수 있어서 주방이 항상 깔끔하게 유지됩니다.\n사이즈도 컴팩트해서 주방 상판 위에 올려두어도 자리 차지를 많이 하지 않고, 그레이스화이트 컬러라 신혼집 인테리어에도 잘 어울려요. 건조 후 분쇄까지 한 번에 되니까 부피가 확 줄어들고, 처리 후 냄새도 거의 없어 만족스럽습니다.\n퇴근 후 설거지하면서 음식물 바로 넣고 버튼만 누르면 끝이라 정말 편하고, 작동 시간도 길지 않아서 부담 없어요. 맞벌이 신혼부부나 음식물 쓰레기 처리 스트레스 줄이고 싶은 분들께 꼭 추천하고 싶은 가전입니다.\n음식물처리기는 신혼부부 필수제품 인 거 같아용\n추천합니다\nNext image"

In [ ]:
f = open("음식물 처리기 카페리뷰.txt","w")
for l in tqdm(href_list):
    driver.get(l)
    time.sleep(2)
    src = driver.find_element(By.ID,'cafe_main')
    driver.switch_to.frame(src)
    time.sleep(2)
    content = preprocess_sentence_kr(driver.find_elements(By.CSS_SELECTOR,'div.se-main-container')[0].text)
    f.write(content)
f.close()

 25%|█████████████████████▊                                                                  | 82/330 [06:37<18:57,  4.59s/it]